In [1]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import GridSearchCV


pd.set_option('display.max_rows', None) # para mostrar todas las filas
pd.set_option('display.max_columns', None) # para mostrar todas las columnas

In [2]:
# Importamos los datos
X_train = pd.read_csv("./DatosPisaSinNan/X_train.csv", sep=',', decimal='.')
y_train = pd.read_csv("./DatosPisaSinNan/y_train.csv", sep=',', decimal='.')
X_val = pd.read_csv("./DatosPisaSinNan/X_val.csv", sep=',', decimal='.')
y_val = pd.read_csv("./DatosPisaSinNan/y_val.csv", sep=',', decimal='.')
X_test = pd.read_csv("./DatosPisaSinNan/X_test.csv", sep=',', decimal='.')
y_test = pd.read_csv("./DatosPisaSinNan/y_test.csv", sep=',', decimal='.')

Pasamos los dataframes a numpy array

In [3]:
feature_names = X_train.columns[:]

X_train = X_train.values
y_train = y_train.values



### Estandarización de datos

Escalamos con los datos de train

In [4]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
#scaler = preprocessing.Normalizer().fit(X_train)
XtrainScaled = scaler.transform(X_train)
XvalScaled = scaler.transform(X_val)
XtestScaled = scaler.transform(X_test)

### SVR

In [5]:
from sklearn.svm import SVR


In [17]:
#parameters=[{'C': [1,10,100,1000],'kernel':['rbf','linear'],'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
parameters=[{'C': [1],'kernel':['rbf', 'linear'],'gamma': [0.1, 0.3, 0.5]}]
modelsvr=SVR()
grid_search=GridSearchCV(estimator=modelsvr,
                         param_grid=parameters,
                         cv=10,
                         n_jobs=-1)
grid_search=grid_search.fit(XtrainScaled,y_train)

In [18]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
0.5784563566338647


In [19]:
#svr=SVR(C= 100, gamma= 0.1, kernel= 'linear')
#svr.fit(XtrainScaled,y_train)

In [20]:
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


ytrainsvr = grid_search.predict(XtrainScaled)
ytestsvr  = grid_search.predict(XtestScaled)

mseTrainModelsvr = mean_squared_error(y_train,ytrainsvr)
mseTestModelsvr = mean_squared_error(y_test,ytestsvr)

print('MSE Modelo svr (train): %0.5g' % mseTrainModelsvr)
print('MSE Modelo svr (test) : %0.5g' % mseTestModelsvr)

print()

maeTrainModelsvr = mean_absolute_error(y_train,ytrainsvr)
maeTestModelsvr  = mean_absolute_error(y_test,ytestsvr)

print('MAE Modelo svr (train): %0.5g' % maeTrainModelsvr)
print('MAE Modelo svr (test) : %0.5g' % maeTestModelsvr)

MSE Modelo svr (train): 2523.2
MSE Modelo svr (test) : 2556.4

MAE Modelo svr (train): 39.694
MAE Modelo svr (test) : 40.27


In [21]:
# Salvamos el modelo en formato joblib

from joblib import dump, load
dump(grid_search, 'modeloSVR_st.joblib') 

#clf = load('modeloSVR_st.joblib') 

['modeloSVR_st.joblib']

### Conclusión al modelo SVR

Hemos obtenido un ajuste relativamente bueno, el mejor de todos los modelos entrenados, y además controlando el overfitting.
